In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re
from pyarrow import csv

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import gc
import os
from sklearn.cluster import DBSCAN
from os import listdir
from os.path import isfile, join
pd.set_option('max_colwidth', None)
from fitter import Fitter, get_common_distributions, get_distributions
import seaborn as sns

In [ ]:
from fitter import Fitter, get_common_distributions, get_distributions

In [ ]:
import seaborn as sns

In [ ]:
dfxw = pd.read_csv('Final_Cluster_300k.csv')

In [ ]:
start = time.time()
df_tweet_lang2 = pd.read_csv("Latest_Tweet_Fake_Spreader_Tweets.csv", lineterminator='\n', dtype=str)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
df_embedding = pd.read_csv('All_Embedding.csv', engine = 'pyarrow')
df_embedding = df_embedding.drop(df_embedding.columns[0:2], axis=1)
end = time.time()
print(end-start)

In [ ]:
len(df_embedding)

# NaMo App Analysis

In [ ]:
df_tweet_lang2['NaMo'] = df_tweet_lang2.Tweet_Text.str.find("via NaMo App")
df_tweet_lang2['NMApp'] = df_tweet_lang2.Tweet_Text.str.find("via NMApp")
df_tweet_lang2['App'] = df_tweet_lang2.Tweet_Text.str.find("App")
df_tweet_lang2['NM-4'] = df_tweet_lang2.Tweet_URL.str.find("nm-4")
df_tweet_lang2['narendramodi'] = df_tweet_lang2.Tweet_URL.str.find("narendramodi.in")

In [ ]:
df_Namo = df_tweet_lang2.loc[df_tweet_lang2.NaMo != -1]
df_NM = df_tweet_lang2.loc[df_tweet_lang2.NMApp != -1]
df_nm4 = df_tweet_lang2.loc[df_tweet_lang2['NM-4'] != -1]
df_naren = df_tweet_lang2.loc[df_tweet_lang2['narendramodi'] != -1]
df_nm4 = df_nm4.dropna(subset = ['NM-4'])
df_naren =df_naren.dropna(subset = ['narendramodi'])

In [ ]:
print(len(df_Namo))
print(len(df_NM))
print(len(df_nm4))
print(len(df_naren))

In [ ]:
df_tweet_lang2.sort_values(by = 'Tweet_Time')

In [ ]:
df_Namo.groupby('User_ID').count()

In [ ]:
df_namo_full = pd.concat([df_Namo, df_NM, df_nm4, df_naren])

In [ ]:
df_namo_full = df_namo_full.drop_duplicates('Tweet_ID')

In [ ]:
df_namo_full = df_namo_full.sort_values(by = 'Tweet_Time', ascending = False).reset_index(drop = True)

In [ ]:
len(df_namo_full)

In [ ]:
df_namo_full.Tweet_ID.astype(int)

In [ ]:
print(len(df_namo_full.groupby('User_ID').count()))


In [ ]:
df_namo_full.columns

In [ ]:
df_namo_full.groupby('Media_Type').count()

In [ ]:
df_namo_full.Tweet_URL.str.replace('www.', '').sample(10)

In [ ]:
df_embedding.Tweet_ID = df_embedding.Tweet_ID.astype(str)

In [ ]:
df_namo_full['Text_without_Link'] = df_namo_full['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_namo_full['Text_without_Link']=df_namo_full['Text_without_Link'].str.replace('(@\w+.*?)',"")

In [ ]:
df_namo_full['Text_without_Link'] = df_namo_full['Text_without_Link'].replace('', np.nan)
df_namo_full = df_namo_full.dropna(subset=['Text_without_Link'])
df_namo_full = df_namo_full.reset_index(drop = True)
df_namo_full['Text_Size'] =df_namo_full.Text_without_Link.apply(lambda x: len(x))
df_namo_full= df_namo_full.reset_index(drop = True)

In [ ]:
len(df_namo_full)

In [ ]:
device = "cuda:0"
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
model.max_seq_length = 350
model.do_lower_case =  'True'

In [ ]:
start = time.time()

emb = model.encode(df_namo_full.Text_without_Link.tolist())
print("Embeddings computed. Shape:", emb.shape)

end = time.time()
print((end - start)/60)

In [ ]:
emb_df = pd.DataFrame(emb)
emb_df['Tweet_ID'] = df_namo_full.Tweet_ID.tolist()
df_embedding_namo = emb_df

In [ ]:
df_embedding_namo.iloc[:,:-1]

In [ ]:
X1 = np.array(df_embedding_namo.iloc[:,:-1])
clustering1 = DBSCAN(eps=1, min_samples=2).fit(X1)

In [ ]:
ew_pd = pd.DataFrame(clustering1.labels_)

In [ ]:
ew_pd['Tweet_ID'] = df_embedding_namo['Tweet_ID']

In [ ]:
ew_pd = ew_pd.rename(columns = {0: 'Cluster_ID'})

In [ ]:
ew_pd.groupby('Cluster_ID').count().reset_index(drop = False).sort_values(by = 'Tweet_ID', ascending = False)

In [ ]:
ew_pd.loc[ew_pd.Cluster_ID == 1342].Tweet_ID.tolist()

In [ ]:
ew_pd

In [ ]:
df_namo_full[df_namo_full.Tweet_ID.isin(ew_pd.loc[ew_pd.Cluster_ID == 1261].Tweet_ID.tolist())].User_ID.unique()

In [ ]:
df_namo_full_total_post = df_namo_full.groupby('User_ID').count().reset_index(drop = False).sort_values('Tweet_ID', ascending = False)[['User_ID', 'Tweet_ID']]

In [ ]:
df_namo_full_total_post.iloc[0:17] 

In [ ]:
df_namo_full_total_post['Post1-10'] = 'False'
df_namo_full_total_post['Post10-100'] = 'False'
df_namo_full_total_post['Post100Above'] = 'False'

In [ ]:
df_namo_full_total_post.loc[(df_namo_full_total_post['Tweet_ID'] >=1) & (df_namo_full_total_post['Tweet_ID'] <=10), 'Post1-10'] = 'True'
df_namo_full_total_post.loc[(df_namo_full_total_post['Tweet_ID'] >=11) & (df_namo_full_total_post['Tweet_ID'] <=100), 'Post10-100'] = 'True' 
df_namo_full_total_post.loc[(df_namo_full_total_post['Tweet_ID'] >=101) & (df_namo_full_total_post['Tweet_ID'] <=100000), 'Post100Above'] = 'True' 

In [ ]:
df_namo_full_total_post.groupby('Post100Above').count()

In [ ]:
ew_pd

In [ ]:
df_namo_full['Cluster_ID'] = ew_pd['Cluster_ID']

In [ ]:
df_acc_check = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_acc_check.User_ID = df_acc_check.User_ID.astype('str')

In [ ]:
df_namo_full_total_post = pd.merge(df_namo_full_total_post, df_acc_check[['User_ID', 'Political', 'Influencer', 'acc_type']], on = 'User_ID')

In [ ]:
df_namo_full_total_post.groupby('Political').count()

In [ ]:
len(df_namo_full_total_post.loc[(df_namo_full_total_post['Post100Above'] == 'True') 
                            & (df_namo_full_total_post['Influencer'] == 'Influencer')
                            & (df_namo_full_total_post['acc_type'] == 'Verified')])

In [ ]:
post_list = ['Post1-10','Post10-100','Post100Above']
acc_type = ['Verified', 'Bot', 'Regular']
pol_inf = ['Political','Influencer']

for i in post_list:
    for j in acc_type:
        print(i)
        print(j)
        print(len(df_namo_full_total_post.loc[(df_namo_full_total_post[i] == 'True') 
                            & (df_namo_full_total_post.acc_type == j)])
             )

In [ ]:
df_namo_full.Tweet_Time = pd.to_datetime(df_namo_full.Tweet_Time, format='%Y-%m-%d %H:%M:%S.%f')
df_namo_full['Date']= df_namo_full.Tweet_Time.dt.date
df_namo_full['Date']= pd.to_datetime(df_namo_full.Date, format='%Y-%m-%d')
df_namo_full['Year']= df_namo_full['Date'].dt.year

In [ ]:
df_namo_full.loc[df_namo_full.Tweet_Lang == 'en'].sample(20)


In [ ]:
df_namo_full.loc[df_namo_full.Cluster_ID == 472]

In [ ]:
df_namo_full.groupby('Year').count().reset_index(drop = False)[['Year', 'Tweet_Time']]

In [ ]:
df_tweet_lang2_URL = df_tweet_lang2.dropna(subset=['Tweet_URL'])

In [ ]:
df_tweet_lang2_URL['NM-4'] = df_tweet_lang2_URL.Tweet_URL.str.find("nm-4")
df_tweet_lang2_URL['narendramodi'] = df_tweet_lang2_URL.Tweet_URL.str.find("narendramodi.in")

In [ ]:
df_namo_full

In [ ]:
df_tweet_lang2_URL.loc[df_tweet_lang2_URL['NM-4'] != -1].sort_values(by = 'Tweet_Time', ascending = False)

In [ ]:
len(df_namo_full_total_post.loc[(df_namo_full_total_post.Political == 'Political') & 
                           (df_namo_full_total_post.acc_type == 'Verified')])

In [ ]:
len(df_namo_full_total_post.loc[(df_namo_full_total_post.Influencer == 'Influencer') & 
                           (df_namo_full_total_post.acc_type == 'Verified')])

In [ ]:
df_namo_full.Tweet_ID.tolist()

In [ ]:
df_namo_full_with_cluster_ID = pd.merge(df_namo_full,ew_pd, on = 'Tweet_ID')

In [ ]:
df_namo_full_with_cluster_ID_onlyhashtag =df_namo_full_with_cluster_ID.loc[df_namo_full_with_cluster_ID.Tweet_HashTags != 'None']

In [ ]:
df_namo_full_with_cluster_ID_onlyhashtag.Total_Hashtags = df_namo_full_with_cluster_ID_onlyhashtag.Total_Hashtags.astype(int)

In [ ]:
df_namo_full_with_cluster_ID_onlyhashtag['Text_without_Link'] = df_namo_full_with_cluster_ID_onlyhashtag['Text_without_Link'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])

In [ ]:
df_namo_full_with_cluster_ID['Text_without_Link'] = df_namo_full_with_cluster_ID['Text_without_Link'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
df_namo_full_with_cluster_ID['Text_without_Link'] = df_namo_full_with_cluster_ID['Text_without_Link'].replace(r'\n',' ', regex=True) 


In [ ]:
pwd

In [ ]:
text_wc = df_namo_full_with_cluster_ID.loc[df_namo_full_with_cluster_ID.Tweet_Lang == 'en'].Text_without_Link.values 

wordcloud_namo = WordCloud(background_color = 'white', max_font_size=50, max_words=300, width=500, height=300).generate(str(text_wc))

plt.imshow(wordcloud_namo)
plt.axis("off")
plt.show()
wordcloud_namo.to_file("first_review.png")

In [ ]:
text_wc[105]

In [ ]:
df_copycat = pd.read_csv('Updated_CopyCat.csv')

In [ ]:
df_copycat.Cat = df_copycat.Cat.astype(str)
df_copycat.CopyCat = df_copycat.CopyCat.astype(str)

In [ ]:
(list(set(df_copycat.Cat)) + list(set(df_copycat.CopyCat)))

In [ ]:
df_copycat_tweet = df_tweet_lang2[df_tweet_lang2.User_ID.isin((list(set(df_copycat.Cat)) + list(set(df_copycat.CopyCat))))]

In [ ]:
df_copycat_tweet ['Text_without_Link'] = df_copycat_tweet['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_copycat_tweet ['Text_without_Link']=df_copycat_tweet['Text_without_Link'].str.replace('(@\w+.*?)',"")
df_copycat_tweet ['Text_without_Link']=df_copycat_tweet['Text_without_Link'].str.replace('(#\w+.*?)',"")
df_copycat_tweet ['Text_without_Link']=df_copycat_tweet['Text_without_Link'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
df_copycat_tweet['Text_without_Link'] = df_copycat_tweet['Text_without_Link'].replace(r'\n',' ', regex=True)

In [ ]:
df_copycat_tweet_eng = df_copycat_tweet.loc[(df_copycat_tweet.Tweet_Lang == 'en')]

In [ ]:
df_copycat_tweet_eng

In [ ]:
text_wc = " ".join(review for review in df_copycat_tweet_eng.loc[df_copycat_tweet_eng.Tweet_Lang == 'en'].Text_without_Link)


In [ ]:
from PIL import Image
cat_mask = np.array(Image.open("cat.png"))

In [ ]:
wordcloud_copycat = WordCloud(background_color = 'white', 
                              stopwords = ['via NaMo', 'amp', 'NaMo App', 'will', 'ग', 'र', 't','s',
                                          'त', 'u', 'many', 'must', 'come', 'even', 'ji'] + list(STOPWORDS),
                              max_font_size=50, 
                              max_words=100, 
                              width=500, 
                              collocations=False, 
                              colormap="brg_r",
                              height=300).generate(str(text_wc))

plt.imshow(wordcloud_copycat, interpolation='bilinear')
plt.axis("off")

wordcloud_copycat.to_file("second_review.png")

In [ ]:
len(wordcloud_copycat.words_)

In [ ]:
wordcloud_copycat.words_

In [ ]:
df_namo_full['Cluster_ID'] = ew_pd['Cluster_ID']

In [ ]:
df_namo_full.loc[df_namo_full.Cluster_ID != -1].sort_values(by = 'Cluster_ID').to_csv('Namo_Analysis.csv')

In [ ]:
df_namo_full['Tweet_Date'] = pd.to_datetime(df_namo_full['Tweet_Time']) 

In [ ]:
df_namo_full['Tweet_Date2'] = df_namo_full['Tweet_Date'].dt.to_period('M')

In [ ]:
df_namo_full['Tweet_Date3'] = df_namo_full['Tweet_Date'].dt.to_period('W')

In [ ]:
df_namo_full['Tweet_Date3'] = pd.to_datetime(df_namo_full['Tweet_Date3'].dt.strftime('%Y-%m-%d'))

In [ ]:
df_namo_full_cluster = df_namo_full.loc[df_namo_full.Cluster_ID != -1]

In [ ]:
df_namo_full_cluster = df_namo_full_cluster.sort_values(by = 'Cluster_ID')

In [ ]:
df_namo_full_cluster.loc[df_namo_full_cluster.Cluster_ID == 19]

In [ ]:
df_namo_full_cluster.Tweet_Date = pd.to_datetime(df_namo_full_cluster.Tweet_Time, errors='coerce')

In [ ]:
df_quest = df_namo_full_cluster.groupby('Cluster_ID').resample('M', on='Tweet_Date').count()

In [ ]:
df_quest = df_quest.rename(columns = {'Cluster_ID': 'Cluster_ID2', 'Tweet_Date':'Tweet2Date'}).reset_index()

In [ ]:
df_quest.groupby('Cluster_ID').sum().iloc[10:20]

In [ ]:
duplicates = df_quest.duplicated()


In [ ]:
df_quest['Number_of_Tweets'] = df_quest['Tweet_Time']

In [ ]:
df_quest_small = df_quest[['Cluster_ID', 'Tweet_Date', 'Number_of_Tweets']]

In [ ]:
df_quest_small['Tweet_Date2' ]= pd.to_datetime(df_quest_small['Tweet_Date'].dt.strftime('%Y-%m'))

In [ ]:
df_quest_cluster_number = df_quest_small.groupby('Cluster_ID').count().reset_index()

In [ ]:
interesting_namo_clusters = df_quest_cluster_number.loc[df_quest_cluster_number.Number_of_Tweets > 1].Cluster_ID.tolist()

In [ ]:
len(df_namo_full_cluster.groupby('Cluster_ID'))

In [ ]:
len(interesting_namo_clusters)

In [ ]:
3996 - 253

In [ ]:
df_quest_small[df_quest_small.Cluster_ID.isin(interesting_namo_clusters)]

In [ ]:
df_quest_small

In [ ]:
df_namo_full[df_namo_full.Cluster_ID.isin(interesting_namo_clusters)].sort_values(by = 'Cluster_ID').sample(20)

In [ ]:
df_namo_full[df_namo_full.Cluster_ID.isin(interesting_namo_clusters)].groupby('Cluster_ID').count().sort_values(by = 'Tweet_ID').iloc[-40:-30]

In [ ]:
df_namo_full.loc[df_namo_full.Cluster_ID == 634].sort_values(by = 'Tweet_Time')

In [ ]:
df_namo_full[df_namo_full.Cluster_ID.isin(interesting_namo_clusters)].groupby('Cluster_ID').count().sort_values(by = 'Tweet_ID').iloc[-30:-20]

In [ ]:
22529-7216

# Namo from original clusters

In [ ]:
df_namo_from_cluster = df_namo_full.Tweet_ID.tolist()

In [ ]:
dfxw_namo = dfxw[dfxw.Tweet_ID.isin(df_namo_from_cluster)]

In [ ]:
dfxw_namo

In [ ]:
dfssw = dfxw_namo.groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
dfssw.loc[dfssw.Tweet_ID != 1]

In [ ]:
l1= dfxw_namo.Tweet_ID.to_list()
l2 =df_namo_full.Tweet_ID.to_list()

In [ ]:
li = l1 + l2

In [ ]:
df_namo_link_check = df_namo_full.Tweet_URL.str.replace('www.', '')

In [ ]:
df_namo_link_check = df_namo_link_check.str.replace('https://', '')

In [ ]:
ew_pd

In [ ]:
df_namo_link_check = df_namo_link_check.str.split('.')

In [ ]:
for i in df_namo_link_check:
    try:
        print(i[0])
    except:
        continue

# Analysis

In [ ]:
dfxw.dtypes

In [ ]:
dfxw.Tweet_ID =dfxw.Tweet_ID.astype(str)

In [ ]:
print(len(dfxw))

In [ ]:
df_tweet_lang2

In [ ]:
df = pd.merge(df_tweet_lang2,dfxw[['Tweet_ID','Cluster_ID', 'Tweet_Link']], on = 'Tweet_ID' ).sort_values(by = 'Cluster_ID')

In [ ]:
df['Text_without_Link'] = df['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['Text_without_Link'] = df['Text_without_Link'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
df['Text_without_Link'] = df['Text_without_Link'].str.replace('(@\w+.*?)',"")

In [ ]:
df['Text_Size'] = df.Text_without_Link.apply(lambda x: len(x))

In [ ]:
df.Tweet_Time = pd.to_datetime(df.Tweet_Time, format='%Y-%m-%d %H:%M:%S.%f')
df['Date']= df.Tweet_Time.dt.date
df['Date']= pd.to_datetime(df.Date, format='%Y-%m-%d')

In [ ]:
df = df.sort_values(['Cluster_ID', 'Date'], ascending=[True, False])

In [ ]:
df_cluster_list = df.groupby('Cluster_ID').count().reset_index(drop = False).Cluster_ID.tolist()

In [ ]:
k = 0
cl_list = []
time_diff_list = []
for i in df_cluster_list:
    df_cluster_specific = df.loc[df.Cluster_ID == i]
    time_diff = df_cluster_specific.iloc[0].Date - df_cluster_specific.iloc[-1].Date
    cl_list.append(i)
    time_diff_list.append(time_diff)
   


In [ ]:
df_time_interval = pd.DataFrame({'Cluster_ID': cl_list, 'Time_Interval':time_diff_list})

In [ ]:
df_time_interval

In [ ]:
df_100 = df[df.Text_Size >= 100]
df_150 = df[df.Text_Size >= 150]
df_200 = df[df.Text_Size >= 200]

In [ ]:
df_100_cluster = df_time_interval[df_time_interval.Cluster_ID.isin(df_100.Cluster_ID)]
df_150_cluster = df_time_interval[df_time_interval.Cluster_ID.isin(df_150.Cluster_ID)]
df_200_cluster = df_time_interval[df_time_interval.Cluster_ID.isin(df_200.Cluster_ID)]

In [ ]:
df_200_cluster

In [ ]:
df_100 = df[df.Text_Size >= 100]
df_150 = df[df.Text_Size >= 150]
df_200 = df[df.Text_Size >= 200]

In [ ]:
df_100.Cluster_ID

In [ ]:
pd.merge(df_100, df_time_interval, on = 'Cluster_ID')

In [ ]:
print(len(df_100))
print(len(df_150))
print(len(df_200))

In [ ]:
print(len(df_100.groupby('Cluster_ID').count()))
print(len(df_150.groupby('Cluster_ID').count()))
print(len(df_200.groupby('Cluster_ID').count()))

In [ ]:
df_cl100 = df_100.groupby('Cluster_ID').count()
df_cl150 = df_150.groupby('Cluster_ID').count()
df_cl200 = df_200.groupby('Cluster_ID').count()

In [ ]:
df10_cl100 = df_cl100.loc[df_cl100.Tweet_ID >= 10].reset_index(drop = False)
df10_cl150 = df_cl150.loc[df_cl150.Tweet_ID >= 10].reset_index(drop = False)
df10_cl200 = df_cl200.loc[df_cl200.Tweet_ID >= 10].reset_index(drop = False)

In [ ]:
print(len(df10_cl100))
print(len(df10_cl150))
print(len(df10_cl200))

In [ ]:
df_time_interval_100 = df_time_interval[df_time_interval.Cluster_ID.isin(df10_cl100.Cluster_ID)]
df_time_interval_150 = df_time_interval[df_time_interval.Cluster_ID.isin(df10_cl150.Cluster_ID)]
df_time_interval_200 = df_time_interval[df_time_interval.Cluster_ID.isin(df10_cl200.Cluster_ID)]
print(len(df_time_interval_100))
print(len(df_time_interval_150))
print(len(df_time_interval_200))

In [ ]:
print(len(df_time_interval_100.loc[df_time_interval_100.Time_Interval >= '7 days']))
print(len(df_time_interval_100.loc[df_time_interval_100.Time_Interval >= '15 days']))
print(len(df_time_interval_100.loc[df_time_interval_100.Time_Interval >= '30 days']))

print(len(df_time_interval_150.loc[df_time_interval_150.Time_Interval >= '7 days']))
print(len(df_time_interval_150.loc[df_time_interval_150.Time_Interval >= '15 days']))
print(len(df_time_interval_150.loc[df_time_interval_150.Time_Interval >= '30 days']))

print(len(df_time_interval_200.loc[df_time_interval_200.Time_Interval >= '7 days']))
print(len(df_time_interval_200.loc[df_time_interval_200.Time_Interval >= '15 days']))
print(len(df_time_interval_200.loc[df_time_interval_200.Time_Interval >= '30 days']))

In [ ]:
df_150.groupby('Media_Type').count()

In [ ]:
df_200.groupby('Media_Type').count()

In [ ]:
df_check = df_time_interval_200.loc[df_time_interval_200.Time_Interval >= '30 days']

In [ ]:
df.loc[df.Cluster_ID == 4176]

In [ ]:
df_check.loc[df_check.Cluster_ID == 2366]

In [ ]:
check_list = df.loc[df.Cluster_ID == 147].head(1)

In [ ]:
check_list

In [ ]:
for i in df_check.Cluster_ID.tolist()[1:]:
    check = df.loc[df.Cluster_ID == i].head(1)
    check_list = pd.concat([check_list, check])

In [ ]:
check_list.iloc[20:30][['Tweet_Link', 'Date', 'Cluster_ID']]

In [ ]:
df_acc_check2 = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_acc_check2 = df_acc_check2.drop(columns = ['Unnamed: 0'])

In [ ]:
df_duplicate_20 = pd.read_csv('Duplicate_More_Than_20.csv')

In [ ]:
df_duplicate_20_acc_un = list(set(df_duplicate_20.Acc1.drop_duplicates().tolist())) + list(set(df_duplicate_20.Acc2.drop_duplicates().tolist()))

In [ ]:
df_duplicate_20_acc_un = list(set(df_duplicate_20_acc_un))

In [ ]:
len(df_duplicate_20_acc_un)

In [ ]:
df_duplicate_20_nodes = pd.DataFrame({'Label' : df_duplicate_20_acc_un})

In [ ]:
df_duplicate_20.Acc1.iloc[0]

In [ ]:
source = []
target = []
dir_type = []

In [ ]:
for i in range(0,len(df_duplicate_20)):
    source.append(df_duplicate_20_nodes.loc[df_duplicate_20_nodes.Label == df_duplicate_20.Acc1.iloc[i]].index[0])
    target.append(df_duplicate_20_nodes.loc[df_duplicate_20_nodes.Label == df_duplicate_20.Acc2.iloc[i]].index[0])
    dir_type.append('Undirected')

In [ ]:
df_edges= pd.DataFrame({'Source': source, 'Target': target, 'Type':dir_type})

In [ ]:
df_edges.loc[df_edges.Target == 359]

In [ ]:
df_edges

In [ ]:
df_100[df_100.Cluster_ID.isin(df10_cl100.Cluster_ID.to_list())].groupby('Tweet_Source').count()

In [ ]:
df10_cl100.Cluster_ID.to_list()

# Cluster Timeline Analysis

In [ ]:
dfxw = dfxw.sort_values(by = ['Cluster_ID', 'Tweet_ID'], ascending = [True, False]).reset_index(drop = True)

In [ ]:
dfxw['DateTime'] =  pd.to_datetime(dfxw['Tweet_Time'])

In [ ]:
dfxw['Date'] = dfxw['DateTime'].dt.date

In [ ]:
cluster_length = dfxw.drop_duplicates('Cluster_ID')

In [ ]:
cluster_list = cluster_length.Cluster_ID.to_list()

In [ ]:
cluster_size = len(dfxw.loc[dfxw.Cluster_ID == 0].Date)

In [ ]:
a = dfxw.loc[dfxw.Cluster_ID == 0].Date.reset_index(drop = True)
new = a[0] - a[cluster_size - 1]

In [ ]:
cluster_date_diff = pd.DataFrame(columns = {'Cluster_ID', 'Date_Diff'})

In [ ]:
cluster_date_diff = cluster_date_diff.rename(columns = {'Date_Diff' : 'Cluster_ID', 'Cluster_ID':'Date_Diff'})

In [ ]:
cluster_date_diff.loc[0] = 0, new 

In [ ]:
cluster_date_diff 

In [ ]:
k = 1
for i in cluster_list[1:]:
    cluster_size = len(dfxw.loc[dfxw.Cluster_ID == i].Date)
    a = dfxw.loc[dfxw.Cluster_ID == i].Date.reset_index(drop = True)
    date_diff = a[0] - a[cluster_size - 1]
    cluster_date_diff.loc[k] = i, date_diff
    k = k+1
    

In [ ]:
cluster_date_diff.to_csv('Cluster_Interval.csv')

In [ ]:
cluster_date_diff['Date_Diff_Int'] = cluster_date_diff['Date_Diff'].dt.days.astype(int)

In [ ]:
cluster_date_diff.Date_Diff_Int = cluster_date_diff['Date_Diff_Int'].abs()

In [ ]:
cluster_date_diff.sort_values(by = 'Date_Diff_Int')

In [ ]:
cluster_date_diff.Date_Diff_Int = cluster_date_diff.Date_Diff_Int + 1

In [ ]:
cluster_date_diff['cdf'] = cluster_date_diff.Date_Diff_Int.rank(method = 'average', pct = True)

In [ ]:
len(cluster_date_diff.loc[cluster_date_diff.Date_Diff_Int < 365])/len(cluster_date_diff)

In [ ]:
cluster_date_diff.Date_Diff_Int.max()/(365/4)

In [ ]:
ax = cluster_date_diff.Date_Diff_Int.plot(kind = 'hist', logy=True,bins=34)
ax.set(xticks=([0, 365, 365*2, 365*3, 365*4, 365*5, 365*6, 365*7, 365*8, 365*9]))

In [ ]:
sns.set_style("whitegrid", {"grid.color": ".95"})
sns.set_context("paper", font_scale = 1.2)
g = sns.displot(data=cluster_date_diff, x="Date_Diff_Int", kind="hist", bins = 34, aspect = 1.2, log = True,)
g.set(xticks=np.arange(0,3285,365))
g.set(xlabel ="Time interval in days", ylabel = "Number of clusters",)
Time_Interval_values = cluster_date_diff["Date_Diff_Int"].values

In [ ]:
from matplotlib.font_manager import fontManager, FontProperties


In [ ]:
f = Fitter(Time_Interval_values,
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm",
                          "halflogistic"]
          )
f.fit()
f.summary()

In [ ]:
f.get_best(method = 'sumsquare_error')

In [ ]:
ax3 = cluster_date_diff.Date_Diff_Int.plot(kind="hist", bins=10)
ax3.set_xlabel("Time Interval in days")

In [ ]:
cluster_date_diff.describe()

In [ ]:
cluster_date_diff.loc[cluster_date_diff.Date_Diff_Int >= 365]

In [ ]:
cluster_date_diff.sort_values('Date_Diff_Int').plot(x = 'Date_Diff_Int', y = 'cdf', legend = False,  xlabel = 'Time Interval', ylabel = 'Fraction of Clusters')

In [ ]:
cluster_date_diff.loc[cluster_date_diff.Date_Diff_Int > 365]

In [ ]:
cluster_date_diff.Date_Diff_Int.describe()

In [ ]:
dfxw.groupby('Cluster_ID').count().reset_index(drop = False).describe()

# Character/Word Analysis

In [ ]:
df_tweet_lang2[df_tweet_lang2.Tweet_ID.isin(dfxw.Tweet_ID.tolist())]

In [ ]:
df_character = pd.merge(df_tweet_lang2, dfxw, on = 'Tweet_ID')

In [ ]:
df_character.columns

In [ ]:
df_character['Text_without_Link'] = df_character['Tweet_Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_character['Text_without_Link'] = df_character['Text_without_Link'].apply(lambda x: re.split('http:\/\/.*', str(x))[0])
df_character['Text_without_Link'] = df_character['Text_without_Link'].str.replace('(@\w+.*?)',"")

In [ ]:
df_character['Text_Size'] = df_character.Text_without_Link.apply(lambda x: len(x))

In [ ]:
df_character.loc[df_character.Text_Size <= 280]

In [ ]:
df_character['Text_Size'].describe()

In [ ]:
bx = df_character.Text_Size.plot.hist(bins=100)

In [ ]:
df_character2 = df_character

In [ ]:
df_character2['Text_without_Link'] = df_character2['Text_without_Link'].str.strip()
df_character2['Text_Size'] = df_character2.Text_without_Link.apply(lambda x: len(x))

In [ ]:
df_character2['word_count'] =  df_character2['Text_without_Link'].str.split().str.len()

In [ ]:
df_character2_filter = df_character2.loc[df_character2.Text_Size >= 20]

In [ ]:
cx = df_character2_filter.loc[df_character2_filter.Text_Size <= 280].word_count.plot.hist(bins=40)

In [ ]:
df_cluster_char = df_character2_filter.groupby('Cluster_ID').mean().reset_index(drop = False)
df_cluster_char = df_cluster_char.loc[df_cluster_char.Text_Size <= 280]

In [ ]:
77/37

In [ ]:
dfxw.loc[dfxw.Cluster_ID == 336]

In [ ]:
df_cluster_char['charPerWord']=df_cluster_char.Text_Size / df_cluster_char.word_count

In [ ]:
df_cluster_char.loc[df_cluster_char.charPerWord > 10]

In [ ]:
df_cluster_char.loc[df_cluster_char.Text_Size >= 203].describe()

In [ ]:
df_cluster_char.describe()

In [ ]:
cx2 = df_cluster_char.loc[df_cluster_char.Text_Size <= 280].Text_Size.plot.hist(bins=20)

In [ ]:
sns.set_style("whitegrid", {"grid.color": ".95"})
sns.set_context("paper", font_scale = 1.2)
g2 = sns.displot(data=df_cluster_char, x="Text_Size", kind="hist", bins = 24, aspect = 1.2, log = False,)
g2.set(xlabel ="Average character length", ylabel = "Number of clusters",)


In [ ]:
cx2 = df_cluster_char.charPerWord.plot.hist(bins=100)

In [ ]:
df_cluster_char.Text_Size.tolist()

In [ ]:
f2 = Fitter(df_cluster_char.Text_Size.tolist(),
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm",]
          )
f2.fit()
f2.summary()

In [ ]:
f3 = Fitter(df_cluster_char.word_count.tolist(),
           distributions=['gamma',
                          'lognorm',
                          "beta",
                          "burr",
                          "norm",]
          )
f3.fit()
f3.summary()

In [ ]:
cx3 = df_cluster_char.loc[df_cluster_char.Text_Size <= 280].word_count.plot.hist(bins=20)

In [ ]:
sns.set_style("whitegrid", {"grid.color": ".95"})
sns.set_context("paper", font_scale = 1.2)
g2 = sns.displot(data=df_cluster_char, x="word_count", kind="hist", bins = 24, aspect = 1.2, log = False,)
g2.set(xlabel ="Average number of words", ylabel = "Number of clusters",)


In [ ]:
df_character2_filter.loc[df_character2_filter.Text_Size <= 280].word_count.describe()

In [ ]:
df_character2_filter.word_count.describe()

In [ ]:
dx = df_character2_filter.loc[df_character2_filter.Text_Size <= 280].Text_Size.plot.hist(bins=100)

In [ ]:
df_character2_filter.loc[df_character2_filter.Text_Size <= 280].Text_Size.describe()

In [ ]:
df_character2_filter.Text_Size.describe()

In [ ]:
df_character2_filter

In [ ]:
df_character2_filter['cdf_text'] = df_character2_filter.loc[df_character2_filter.Text_Size <= 280].Text_Size.rank(method = 'average', pct = True)

In [ ]:
df_character2_filter['cdf_word'] = df_character2_filter.loc[df_character2_filter.Text_Size <= 280].word_count.rank(method = 'average', pct = True)

In [ ]:
df_character2_filter.sort_values('Text_Size').plot(x = 'Text_Size', y = 'cdf_text', legend = False, xlim = [10,300], xlabel = 'Number of Characters', ylabel = 'Fraction of Posts')

In [ ]:
df_character2_filter.sort_values('word_count').plot(x = 'word_count', y = 'cdf_word', legend = False, xlabel = 'Number of Words', ylabel = 'Fraction of Posts')

# Multimedia Analysis

In [ ]:
df_multimedia = df_character.dropna(subset=['Media_Type'])

In [ ]:
df_multimedia.groupby('Cluster_ID').count()

In [ ]:
df_multimedia.drop_duplicates('Cluster_ID').groupby('Media_Type').count()

# Cluster Basic

In [ ]:
dfxw.groupby('Cluster_ID').count().reset_index(drop = False).describe()

In [ ]:
dfxw.groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
dfxw2 = dfxw.groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
dfxw2['Tweet_in_Cluster'] = dfxw2.Tweet_ID.rank(method = 'average', pct = True)

In [ ]:
len(dfxw2.loc[dfxw2.Tweet_ID <10])/len(dfxw2)

In [ ]:
sx = dfxw2.Tweet_ID.plot.hist(bins = 40, logy = True)

In [ ]:
dfxw2.sort_values('Tweet_ID').plot(x = 'Tweet_ID', y = 'Tweet_in_Cluster', legend = False, xlabel = 'Number of Tweets', ylabel = 'Fraction of Clusters',)

In [ ]:
dfxw2

In [ ]:
dfxw.groupby('Cluster_ID').count().reset_index(drop = False).loc[dfxw.groupby('Cluster_ID').count().reset_index(drop = False).Tweet_ID >= 10]

In [ ]:
dfxw.groupby('Cluster_ID').count().reset_index(drop = False)

In [ ]:
df.columns

In [ ]:
df_tweet_lang2